In [1]:
# OMP_NUM_THREADS=16 CUDA_VISIBLE_DEVICES=0 python demo.py 
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "3, 4"
os.environ["TOKENIZERS_PARALLELISM"] = "False"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
from modeling_mixtral import MixtralForCausalLM
from transformers import AutoTokenizer
import torch
import torch.nn as nn
from typing import Optional
import json

with open('../path.json', 'r') as f:
    path = json.load(f)
    model_name = path['mixtral']

save_dir = './hqqsaved'
backend       = "bitblas" #'torchao_int4' #"torchao_int4" (4-bit only) or "gemlite" (4-bit + 2-bit)
dtype = torch.bfloat16 if backend=="torchao_int4" else torch.float16
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='./')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

### HQQ量化
from hqq.core.quantize import *
from hqq.models.hf.mixtral import MixtralPatch
import transformers
from hqq.models.base import BaseHQQModel
from accelerate import init_empty_weights

class BaseHQQHFModel(BaseHQQModel):
    # Save model architecture
    @classmethod
    def cache_model(cls, model, save_dir):
        model.config.save_pretrained(save_dir)

    # Create empty model from config
    @classmethod
    def create_model(cls, save_dir, kwargs):
        model_kwargs = {}
        for key in ["attn_implementation"]:
            if key in kwargs:
                model_kwargs[key] = kwargs[key]

        config = transformers.AutoConfig.from_pretrained(
            cls.get_config_file(save_dir)
        )

        with init_empty_weights():
            model = MixtralForCausalLM._from_config(config, **model_kwargs)

        return model

class MixtralHQQ(MixtralPatch, BaseHQQHFModel):
    pass
 

llm = MixtralHQQ.from_quantized(save_dir, compute_dtype=dtype, device='cuda:0', use_cache=True)
HQQLinear.set_backend(HQQBackend.PYTORCH)


# #Optimize
from hqq.utils.patching import prepare_for_inference
prepare_for_inference(llm, backend=backend, verbose=True)

100%|██████████| 32/32 [00:00<00:00, 389.97it/s]


NameError: name 'RunTimeError' is not defined

In [ ]:
import json
from datasets import load_dataset, Dataset
from transformers import GenerationConfig

input_length = 8
MAX_LENGTH = input_length
output_length = 16
device_id = 0
test_samples = 1

def preprocess_data(data, tokenizer):
	# 使用 tokenizer 将文本数据转换为模型输入
	inputs = tokenizer(data, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
	inputs["labels"] = inputs.input_ids.clone()
	return inputs

generated_all, decode_time, prefill_time = 0, 0, 0
# print("max output length is {}".format(output_length))
text = "The future of AI is here, and "

inputs = preprocess_data(text, tokenizer)
# # 测试时间
# start_event = torch.cuda.Event(enable_timing=True)
# end_event = torch.cuda.Event(enable_timing=True)

# 开始计时
# torch.cuda.synchronize()
# start_event.record()

# 前向传播
with torch.no_grad():
    output = llm.generate(
        input_ids=inputs["input_ids"].cuda(device_id),
        attention_mask=inputs["attention_mask"].cuda(device_id),
        max_length=input_length + output_length,  # 总长度为输入长度 + 输出长度
        generation_config=GenerationConfig(do_sample=False),
        pad_token_id=tokenizer.pad_token_id, 
        # cache_implementation="static" ## moe not support
    )

# 结束计时
# end_event.record()
# torch.cuda.synchronize()

# 计算时间
# elapsed_time = start_event.elapsed_time(end_event) / 1000  # 转换为秒
# decode_time += elapsed_time
# print(f"Generated length: {len(output[0]) - input_length}", f"Time taken: {elapsed_time:.2f} s")
# print(output)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

# generated_all += (len(output[0]) - input_length -1)

# timepertoken = (decode_time) / (generated_all)
# print("decode phase speed:", '{:.4f}'.format(1/timepertoken) , ' token/s')

### 先都加载到cpu上

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3, 4"
os.environ["TOKENIZERS_PARALLELISM"] = "False"
from modeling_mixtral import MixtralForCausalLM
from transformers import AutoTokenizer
import torch
import torch.nn as nn
from typing import Optional
import json

with open('../path.json', 'r') as f:
    path = json.load(f)
    model_name = path['mixtral']
    # threshold_path = path[threshold_path_name]

with open("../quantize/device_map.json", "r") as f:
    device_map = json.load(f)

dtype = torch.float16
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='./')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

### HQQ量化
from hqq.core.quantize import *
from hqq.models.hf.mixtral import MixtralPatch
import transformers
from hqq.models.base import BaseHQQModel
from accelerate import init_empty_weights

class BaseHQQHFModel(BaseHQQModel):
    # Save model architecture
    @classmethod
    def cache_model(cls, model, save_dir):
        model.config.save_pretrained(save_dir)

    # Create empty model from config
    @classmethod
    def create_model(cls, save_dir, kwargs):
        model_kwargs = {}
        for key in ["attn_implementation"]:
            if key in kwargs:
                model_kwargs[key] = kwargs[key]

        config = transformers.AutoConfig.from_pretrained(
            cls.get_config_file(save_dir)
        )

        with init_empty_weights():
            model = MixtralForCausalLM._from_config(config, **model_kwargs)

        return model

class MixtralHQQ(MixtralPatch, BaseHQQHFModel):
    pass
 
save_dir = './hqq4'
llm = MixtralHQQ.from_quantized(save_dir, compute_dtype=dtype, device='cuda:0', use_cache=True)
HQQLinear.set_backend(HQQBackend.PYTORCH)

# backend       = "gemlite" #'torchao_int4' #"torchao_int4" (4-bit only) or "gemlite" (4-bit + 2-bit)
# # #Optimize
# from hqq.utils.patching import prepare_for_inference
# prepare_for_inference(llm, backend=backend, verbose=True)
# # Load GemLite cache
# if(backend == 'gemlite'):
# 	import gemlite
# 	gemlite.core.GEMLITE_TRITON_RESTRICT_M = True
# 	gemlite.core.GemLiteLinear.load_config('/tmp/gemlite_config.json')

/home/bcds/.conda/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/bcds/venv/dilab/floe/hqq/hqq/models/base.py:251: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you sta

In [5]:
import torch 

def check_valid(x, W, quant_linear, tol=1e-3):
    y_ref = torch.matmul(x, W.T)
    # print(y_ref)
    y_q   = quant_linear(x)
    # print(y_q)
    print((y_ref - y_q).abs().mean())
    assert (y_ref - y_q).abs().mean() < tol, 'Outputs do not match'

############################################################################################
from hqq.core.quantize import HQQLinear, BaseQuantizeConfig

in_features, out_features = 4096*4, 4096*2
#W_nbits, group_size = 8, in_features 
W_nbits, group_size = 2, 128 
#W_nbits, group_size = 2, 128

linear       = torch.nn.Linear(in_features=in_features, out_features=out_features, bias=False, device='cuda:0')
quant_config = BaseQuantizeConfig(nbits=W_nbits, group_size=group_size, quant_zero=False, quant_scale=False, axis=1)
hqq_layer    = HQQLinear(linear, quant_config=quant_config, compute_dtype=torch.float16, device='cuda:0', del_orig=False) 

orig_shape   = (out_features, in_features)
W            = hqq_layer.dequantize().reshape(orig_shape).to(torch.float16)
############################################################################################

from gemlite.core import GemLiteLinearTriton, DType
gemlite_linear = GemLiteLinearTriton(W_nbits, 
                                    group_size=group_size, 
                                    in_features=in_features, 
                                    out_features=out_features, 
                                    input_dtype=DType.FP16, 
                                    output_dtype=DType.FP16)

W_q           = hqq_layer.unpack(dtype=torch.uint8).view(orig_shape)
scales        = hqq_layer.meta['scale']
zeros         = hqq_layer.meta['zero']
gemlite_linear.pack(W_q, scales, zeros, None)

import random
random.seed(42)
x = torch.randn((1, in_features), dtype=torch.float16, device='cuda:0')/10.

check_valid(x, W, gemlite_linear)

tensor(0.0503, device='cuda:0', dtype=torch.float16)


AssertionError: Outputs do not match

In [2]:
def prepare_for_compile(model):
    # model.config.use_cache = True
    # model.generation_config.cache_implementation = "static"
    # model.eval()
    
    torch._dynamo.config.inline_inbuilt_nn_modules = False #torch 2.5.0 fix

    forward_compiled = torch.compile(
        model.forward, mode="reduce-overhead", fullgraph=True
    )

    model.forward = forward_compiled

### cuda graph is empty ???
for i in range(len(llm.model.layers)):
    for j in range(len(llm.model.layers[0].block_sparse_moe.experts)):
        prepare_for_compile(llm.model.layers[i].block_sparse_moe.experts[j].w3)

In [ ]:
import json
from datasets import load_dataset, Dataset
from transformers import GenerationConfig

input_length = 1
MAX_LENGTH = input_length
output_length = 32
device_id = 0
test_samples = 1

def preprocess_data(data, tokenizer):
	# 使用 tokenizer 将文本数据转换为模型输入
	inputs = tokenizer(data, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
	inputs["labels"] = inputs.input_ids.clone()
	return inputs

generated_all, decode_time, prefill_time = 0, 0, 0
# print("max output length is {}".format(output_length))
text = "The future of AI is here, and "

inputs = preprocess_data(text, tokenizer)

# 前向传播
for i in range(2):
    with torch.no_grad():
        output = llm.generate(
            input_ids=inputs["input_ids"].cuda(device_id),
            attention_mask=inputs["attention_mask"].cuda(device_id),
            max_length=input_length + output_length,  # 总长度为输入长度 + 输出长度
            generation_config=GenerationConfig(do_sample=False),
            pad_token_id=tokenizer.pad_token_id, 
            # cache_implementation="static" ## moe not support
        )

#### 测试一个正常输出

In [ ]:
import json
from datasets import load_dataset, Dataset
from transformers import GenerationConfig

input_length = 8
MAX_LENGTH = input_length
output_length = 16
device_id = 0
test_samples = 1

def preprocess_data(data, tokenizer):
	# 使用 tokenizer 将文本数据转换为模型输入
	inputs = tokenizer(data, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
	inputs["labels"] = inputs.input_ids.clone()
	return inputs

generated_all, decode_time, prefill_time = 0, 0, 0
# print("max output length is {}".format(output_length))
text = "The future of AI is here, and "

inputs = preprocess_data(text, tokenizer)
# # 测试时间
# start_event = torch.cuda.Event(enable_timing=True)
# end_event = torch.cuda.Event(enable_timing=True)

# 开始计时
# torch.cuda.synchronize()
# start_event.record()

# 前向传播
with torch.no_grad():
    output = llm.generate(
        input_ids=inputs["input_ids"].cuda(device_id),
        attention_mask=inputs["attention_mask"].cuda(device_id),
        max_length=input_length + output_length,  # 总长度为输入长度 + 输出长度
        generation_config=GenerationConfig(do_sample=False),
        pad_token_id=tokenizer.pad_token_id, 
        # cache_implementation="static" ## moe not support
    )

# 结束计时
# end_event.record()
# torch.cuda.synchronize()

# 计算时间
# elapsed_time = start_event.elapsed_time(end_event) / 1000  # 转换为秒
# decode_time += elapsed_time
# print(f"Generated length: {len(output[0]) - input_length}", f"Time taken: {elapsed_time:.2f} s")
# print(output)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

# generated_all += (len(output[0]) - input_length -1)

# timepertoken = (decode_time) / (generated_all)
# print("decode phase speed:", '{:.4f}'.format(1/timepertoken) , ' token/s')

In [4]:
import json
from datasets import load_dataset, Dataset
from transformers import GenerationConfig
from gemlite.helper import A16Wn

input_length = 32
MAX_LENGTH = input_length
output_length = 20
test_samples = 5
device_id = 0

with open("../path.json", "r") as f:
    paths = json.load(f)
    fineweb_path = paths["fineweb"]

def preprocess_data(data, tokenizer):
	# 使用 tokenizer 将文本数据转换为模型输入
	inputs = tokenizer(data, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
	inputs["labels"] = inputs.input_ids.clone()
	return inputs

filt_type = fineweb_path.split('.')[-1]
fineweb = load_dataset(filt_type, data_files=fineweb_path) #726000
fineweb_text = fineweb['train']['text']

# 预热（避免第一次运行时的额外开销）
# for text in fineweb_text[:5] :
#     inputs = preprocess_data(text, tokenizer)
#     with torch.no_grad():
#         output = llm(input_ids=inputs["input_ids"].cuda(device_id), attention_mask=inputs["attention_mask"].cuda(device_id))

generated_all = 0
prefill_time, decode_time = 0, 0
reloaded_experts = 0
print("output length is {}".format(output_length))
for text in fineweb_text[2:2+test_samples] :
    inputs = preprocess_data(text, tokenizer)

    # # 测试时间
    # start_event = torch.cuda.Event(enable_timing=True)
    # end_event = torch.cuda.Event(enable_timing=True)

    # # 开始计时
    # torch.cuda.synchronize()
    # start_event.record()
    print(tokenizer.batch_decode(inputs["input_ids"], skip_special_tokens=True))
    # 前向传播
    with torch.no_grad():
        output = llm.generate(
            input_ids=inputs["input_ids"].cuda(device_id),
            attention_mask=inputs["attention_mask"].cuda(device_id),
            max_length=input_length + output_length,  # 总长度为输入长度 + 输出长度
            generation_config=GenerationConfig(do_sample=True),
            pad_token_id=tokenizer.pad_token_id, 
            # cache_implementation="static" ## moe not support
        )

    # 结束计时
    # end_event.record()
    # torch.cuda.synchronize()

    # # 计算时间
    # elapsed_time = start_event.elapsed_time(end_event) / 1000  # 转换为秒
    # decode_time += elapsed_time
    # print(f"Generated length: {len(output[0]) - input_length}", f"Time taken: {elapsed_time:.2f} s,")
    # print(output)
    print(tokenizer.batch_decode(output, skip_special_tokens=True))

    # generated_all += (len(output[0]) - input_length - 1)

# timepertoken = (decode_time) / (generated_all)
# print("decode phase speed:", '{:.4f}'.format(1/timepertoken) , 'token/s')
# print("the number of reloaded experts per token:", '{:.3f}'.format(reloaded_experts / generated_all))

output length is 20
['How do you get HIV?\nHIV can be passed on when infected bodily fluid, such as blood or semen, is passed into an un']
["How do you get HIV?\nHIV can be passed on when infected bodily fluid, such as blood or semen, is passed into an uninfected person's bloodstream. The main way the HIV virus is transmitted is by"]
['CTComms sends on average 2 million emails monthly on behalf of over 125 different charities and not for profits.\nTake the complexity']
['CTComms sends on average 2 million emails monthly on behalf of over 125 different charities and not for profits.\nTake the complexity of working with a CRM system, add on top of that the use of A/B split']
["Hold the salt: UCLA engineers develop revolutionary new desalination membrane\nProcess uses atmospheric pressure plasma to create filtering 'brush"]
["Hold the salt: UCLA engineers develop revolutionary new desalination membrane\nProcess uses atmospheric pressure plasma to create filtering 'brush'\n\nLos Angeles – 

In [5]:
import json
from datasets import load_dataset, Dataset
from transformers import GenerationConfig

input_length = 1
MAX_LENGTH = input_length
output_length = 2
test_samples = 4

with open("../path.json", "r") as f:
    paths = json.load(f)
    fineweb_path = paths["fineweb"]

def preprocess_data(data, tokenizer):
	# 使用 tokenizer 将文本数据转换为模型输入
	inputs = tokenizer(data, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
	inputs["labels"] = inputs.input_ids.clone()
	return inputs

fineweb = load_dataset("parquet",data_files=fineweb_path) #726000
fineweb_text = fineweb['train']['text'][:test_samples] 

print("output length is {}".format(output_length))
text = fineweb_text[0]
inputs = preprocess_data(text, tokenizer)

# cached_mlp.clear_load_from_cpu_stats()
with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ]
) as p:
    # 前向传播
    with torch.no_grad():
        output = llm.generate(
            input_ids=inputs["input_ids"].cuda(),
            attention_mask=inputs["attention_mask"].cuda(),
            max_length=input_length + output_length,  # 总长度为输入长度 + 输出长度
            generation_config=GenerationConfig(do_sample=False),
            pad_token_id=tokenizer.eos_token_id
        )
print(p.key_averages().table(
    sort_by="self_cpu_time_total", row_limit=-1))
p.export_chrome_trace("./compile.json")

output length is 2


/home/bcds/.conda/envs/llm/lib/python3.9/site-packages/torch/cuda/graphs.py:84: UserWarning: The CUDA Graph is empty. This usually means that the graph was attempted to be captured on wrong device or stream. (Triggered internally at ../aten/src/ATen/cuda/CUDAGraph.cpp:208.)
  super().capture_end()


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                  Torch-Compiled Region        77.45%        2.833s        78.41%        2.869s       5.603ms       3.038ms         5.41%      16.160ms      31.563us           512  
                       Scheduler.codegen (dynamo_timed)         9.00%     329.124ms         9.00%     329.124ms     164.562ms       0.000us         0.00%       0.000us       0.000us             2  
         